In [ ]:
#not sure what this does
!pip install mysql-connector-python

In [ ]:
#import relevant libs
import json
import pandas as pd
import mysql.connector
from mysql.connector import errorcode
from datetime import date, datetime, timedelta
import requests

In [ ]:
api_key = 'pKqbMVplJJtJ_GAnrsNEcGBdJLIesOu-fBmpWdJVPhiWSmf_Va3k50VHG3AsxKQTrNo3kaL2HCrw1bGzND0GMWMKzGIoFxGy6zgiWBz1ysTau_hbTd_yb_BXn_MpXnYx'
client_id = 'H1K0c1q-MDYv5bA0ZgZ4XA'
term = 'Greek'
location = 'Astoria NY'
SEARCH_LIMIT = 50
url_params = {'term': term.replace(' ', '+'), 'location': location.replace(' ', '+'), 'limit': SEARCH_LIMIT, 'offset' : None}

In [ ]:
#Yelp connection function
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    return response

In [ ]:
#Parses data into lists which we then coerce into a list of tuples (exceptions made for missing vals)
def parse_results(results):
    r = results.json()
    id_list = []
    alias_list = []
    name_list = []
    review_count_list = []
    rating_list = []
    price_list = []
    final = []
    for i in range(len(r['businesses'])):
        if 'id' in r['businesses'][i].keys():
            id_list.append(r['businesses'][i]['id'])
        else:
            id_list.append('n/a')
        if 'alias' in r['businesses'][i].keys():
            alias_list.append(r['businesses'][i]['alias'])
        else:
            alias_list.append('n/a')
        if 'name' in r['businesses'][i].keys():
            name_list.append(r['businesses'][i]['name'])
        else:
            name_list.append('n/a')
        if 'review_count' in r['businesses'][i].keys():
            review_count_list.append(r['businesses'][i]['review_count'])
        else:
            review_count_list.append('n/a')
        if 'rating' in r['businesses'][i].keys():
            rating_list.append(r['businesses'][i]['rating'])
        else:
            rating_list.append('n/a')
        if 'price' in r['businesses'][i].keys():
            price_list.append(len(r['businesses'][i]['price']))
        else:
            price_list.append('n/a')
        
        final.append((id_list[i], alias_list[i], name_list[i], review_count_list[i], rating_list[i], price_list[i]))
    return final

In [ ]:
#Database name initialized
yelp_db = "Yelp_db"

In [ ]:
#AWS server/db connection
cnx = mysql.connector.connect(
    host = "database-1.c3cnzogwpqkc.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "Psbz67j00",
    port = 3306,
    database = 'Yelp_db'
)
cur = cnx.cursor()

In [ ]:
#Test cursor connection
cur

In [ ]:
#Function to create our database
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
#Call function to actually create the database
create_database(cur, yelp_db)

In [ ]:
#Code to create tables in the database we just created
DB_NAME = 'Yelp_db'

TABLES = {}
TABLES['businesses'] = (
    "CREATE TABLE businesses ("
    "  id varchar(50) NOT NULL,"
    "  alias varchar(50) NOT NULL,"
    "  name varchar(50) NOT NULL,"
    "  review_count int(10) NOT NULL,"
    "  rating float(10) NOT NULL,"
    "  price varchar(10) NOT NULL"
    " ) ENGINE=InnoDB")

TABLES = {}
TABLES['reviews'] = (
    "CREATE TABLE reviews ("
    "  id varchar(50) NOT NULL,"
    "  text varchar(250) NOT NULL,"
    "  time varchar(50)"
    ") ENGINE=InnoDB")

In [ ]:
#Loops through to confirm whether tables have been created as we desired
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cur.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

#cur.close()
#cnx.close()

In [ ]:
#re-establish connection with AWS
cnx = mysql.connector.connect(
    host = "database-1.c3cnzogwpqkc.us-east-2.rds.amazonaws.com",
    user = "admin",
    passwd = "Psbz67j00",
    port = 3306,
    database = 'Yelp_db'
)
cur = cnx.cursor()

In [ ]:
#Function to insert the values into our database
def db_insert(connection, cursor, parsed_results):
    add_business = ("INSERT INTO businesses "
               "(id, alias, name, review_count, rating, price) "
               "VALUES (%s, %s, %s, %s, %s, %s)")
    cursor.executemany(add_business, parsed_results)
    cnx.commit()

In [ ]:
# create a variable  to keep track of which result you are in. 
our_count = 0

#set up a while loop to go through and grab the result 
while our_count < 1000: 
    url_params['offset'] = our_count
    results = yelp_call(url_params, api_key)
    tup = parse_results(results)
    db_insert(cnx, cur, tup)
    our_count += 50

In [ ]:
cur.execute("""SELECT id FROM businesses;""")
business_ids = cur.fetchall()
print(business_ids)

In [ ]:
business_set = set(business_ids)
type(business_set)

In [ ]:
biz_list = []
for biz in business_set:
    biz_list.append(biz[0])
print(biz_list)

In [ ]:
def yelp_review_call(results):
    data = []
    review_list = []
    groups_of_3_reviews = []
    for element in results:
        response = requests.get('https://api.yelp.com/v3/businesses/{}/reviews'.format(element), headers={'Authorization': 'Bearer {}'.format(api_key)})
        data.append(response.json())
    for item in data:
        review_list.append(item['reviews'])
    for i in review_list:
        if len(i) == 3:
            groups_of_3_reviews.append([(i[0]['text'], i[0]['time_created']), (i[1]['text'], i[1]['time_created']), (i[2]['text'], i[2]['time_created'])])
        elif len(i) == 2:
            groups_of_3_reviews.append([(i[0]['text'], i[0]['time_created']), (i[1]['text'], i[1]['time_created'])])
        elif len(i) == 1:
            groups_of_3_reviews.append([(i[0]['text'], i[0]['time_created'])])
        else:
            groups_of_3_reviews.append([('No recent reviews', 'n/a')])
    return groups_of_3_reviews
revs = yelp_review_call(biz_list)

In [ ]:
print(biz_list)

In [ ]:
print(revs)

In [ ]:
print

In [ ]:
def combo(ids, groups):
    id_list = []
    text_list = []
    time_list = []
    count = 0
    combo_list = []
    for i in groups:
        if len(i) == 3:
            id_list.append(ids[count])
            id_list.append(ids[count])
            id_list.append(ids[count])
            text_list.append(i[0][0])
            text_list.append(i[1][0])
            text_list.append(i[2][0])
            time_list.append(i[0][1])
            time_list.append(i[1][1])
            time_list.append(i[2][1])
        elif len(i) == 2:
            id_list.append(ids[count])
            id_list.append(ids[count])
            text_list.append(i[0][0])
            text_list.append(i[1][0])
            time_list.append(i[0][1])
            time_list.append(i[1][1])
        elif len(i) == 1:
            id_list.append(ids[count])
            text_list.append(i[0][0])
            time_list.append(i[0][1])
        else:
            id_list.append(ids[count])
            text_list.append('No recent reviews')
            time_list.append('n/a')
        count += 1
    for i in range(len(id_list)):
        combo_list.append((id_list[i], text_list[i], time_list[i]))
    return combo_list

In [ ]:
combo_result = combo(biz_list, revs)
print(combo_result)

In [ ]:
#Function to insert the values into our review database
def db_insert(connection, cursor, new_parsed_results):
    add_business = ("INSERT INTO reviews "
               "(id, text, time) "
               "VALUES (%s, %s, %s)")
    cursor.executemany(add_business, new_parsed_results)
    cnx.commit()

In [ ]:
db_insert(cnx, cur, combo_result)